In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression

# ============================================================================
# STEP 1: DATA CLEANING
# ============================================================================
print("STEP 1: LOADING AND CLEANING DATA...")
print("=" * 80)

# Load raw data
crop = pd.read_csv('datafile.csv')
crop1 = pd.read_csv('datafile (1).csv')
crop2 = pd.read_csv('datafile (2).csv')
crop3 = pd.read_csv('datafile (3).csv')
produce = pd.read_csv('produce.csv')

# Clean crop
crop.dropna(inplace=True)
crop.columns = crop.columns.str.strip()
crop.to_csv('cleaned_crop.csv', index=False)

# Clean crop1
crop1 = crop1[crop1["Yield (Quintal/ Hectare) "] < 68]
crop1 = crop1[crop1["Cost of Cultivation (`/Hectare) A2+FL"] < 30000]
crop1 = crop1[crop1["Cost of Cultivation (`/Hectare) C2"] < 30000]
crop1 = crop1[crop1["Cost of Production (`/Quintal) C2"] < 3000]
crop1 = crop1[crop1["Yield (Quintal/ Hectare) "] < 25]
crop1.reset_index(inplace=True, drop=True)
crop1.columns = crop1.columns.str.strip()
crop1.to_csv('cleaned_crop1.csv', index=False)

# Clean crop2
crop2 = crop2.drop(index=[0, 9, 13, 14, 16, 15, 33, 38, 47])
crop2.reset_index(inplace=True, drop=True)
crop2.columns = crop2.columns.str.strip()
crop2.to_csv('cleaned_crop2.csv', index=False)

# Clean crop3
crop3 = crop3.drop(columns=["Unnamed: 4"])
crop3["Variety"] = crop3["Variety"].str.split().str[0]
crop3["Season/ duration in days"] = pd.to_numeric(crop3["Season/ duration in days"], errors='coerce')
crop3.dropna(inplace=True)
crop3.reset_index(inplace=True, drop=True)
crop3.columns = crop3.columns.str.strip()
crop3.to_csv('cleaned_crop3.csv', index=False)

# Clean produce
produce.drop(columns=[" 3-1993", " 3-1994", " 3-1995", " 3-1996", " 3-1997",
                      " 3-1998", " 3-1999", " 3-2000", " 3-2001", " 3-2002",
                      " 3-2003", " 3-2004"], inplace=True)
produce.dropna(inplace=True)
produce["Unit"] = (produce["Unit"] == "Ton mn")
produce = produce[produce["Unit"]]
produce["Unit"] = "Ton mn"
produce.reset_index(drop=True, inplace=True)
produce["Frequency"] = produce["Frequency"].str.split(",").str[0]
produce["Particulars"] = produce["Particulars"].str.split(" ").str[3:].str.join("")
produce = produce.rename(columns={"Particulars": "Crop"})
produce.columns = produce.columns.str.strip()
produce.to_csv('cleaned_produce.csv', index=False)

print("Data cleaning completed!")

# ============================================================================
# STEP 2: MERGING DATASETS
# ============================================================================
print("\n" + "=" * 80)
print("STEP 2: MERGING DATASETS...")
print("=" * 80)

# Reload cleaned data
crop = pd.read_csv('cleaned_crop.csv')
crop1 = pd.read_csv('cleaned_crop1.csv')
crop2 = pd.read_csv('cleaned_crop2.csv')
crop3 = pd.read_csv('cleaned_crop3.csv')
produce = pd.read_csv('cleaned_produce.csv')

# Strip whitespace from columns
for df in [crop, crop1, crop2, crop3, produce]:
    df.columns = df.columns.str.strip()

# Check what columns exist
print("Available columns in each dataset:")
print(f"crop: {crop.columns.tolist()}")
print(f"crop1: {crop1.columns.tolist()}")
print(f"crop2: {crop2.columns.tolist()}")
print(f"crop3: {crop3.columns.tolist()}")
print(f"produce: {produce.columns.tolist()}")

# Start merging - use 'Crop' column if it exists, otherwise concatenate
merged_df = produce.copy()

if 'Crop' in crop.columns:
    merged_df = merged_df.merge(crop, on='Crop', how='left', suffixes=('', '_crop'))
    print(f"\nMerged with crop. Shape: {merged_df.shape}")

if 'Crop' in crop1.columns:
    merged_df = merged_df.merge(crop1, on='Crop', how='left', suffixes=('', '_crop1'))
    print(f"Merged with crop1. Shape: {merged_df.shape}")

if 'Crop' in crop2.columns:
    merged_df = merged_df.merge(crop2, on='Crop', how='left', suffixes=('', '_crop2'))
    print(f"Merged with crop2. Shape: {merged_df.shape}")

if 'Crop' in crop3.columns:
    merged_df = merged_df.merge(crop3, on='Crop', how='left', suffixes=('', '_crop3'))
    print(f"Merged with crop3. Shape: {merged_df.shape}")

print(f"\nFinal merged shape: {merged_df.shape}")

# ============================================================================
# STEP 3: HANDLE MISSING VALUES
# ============================================================================
print("\n" + "=" * 80)
print("STEP 3: HANDLING MISSING VALUES...")
print("=" * 80)

print(f"Missing values before cleaning:")
missing_counts = merged_df.isnull().sum()
print(missing_counts[missing_counts > 0])

merged_clean = merged_df.copy()

# Drop columns that have more than 90% missing values
initial_cols = merged_clean.shape[1]
missing_threshold = 0.9 * len(merged_clean)
merged_clean.dropna(axis=1, thresh=len(merged_clean) - missing_threshold, inplace=True)
dropped_cols = initial_cols - merged_clean.shape[1]
print(f"\nDropped {dropped_cols} columns with >90% missing values.")

# Fill numeric columns with median
numeric_cols = merged_clean.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if merged_clean[col].isnull().any(): # Only fill if there are NaNs
        median_val = merged_clean[col].median()
        merged_clean[col].fillna(median_val, inplace=True)

# Fill categorical columns with mode
categorical_cols = merged_clean.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if merged_clean[col].isnull().any(): # Only fill if there are NaNs
        mode_val = merged_clean[col].mode()
        fill_val = mode_val[0] if len(mode_val) > 0 else 'Unknown'
        merged_clean[col].fillna(fill_val, inplace=True)

print(f"Rows after filling NaNs in columns: {len(merged_clean)}")

# Verify no NaN remains at the column level
remaining_nan = merged_clean.isnull().sum().sum()
print(f"\nTotal missing values after column-wise cleaning: {remaining_nan}")

# Drop any remaining rows with NaN values (should be minimal after column dropping and filling)
if remaining_nan > 0:
    print("Warning: Some NaN values still remain. Dropping those rows...")
    merged_clean.dropna(inplace=True)
    print(f"Final rows after dropping remaining NaNs: {len(merged_clean)}")

# Ensure there is still enough data after cleaning
if len(merged_clean) == 0:
    print("ERROR: Merged and cleaned DataFrame is empty. Cannot proceed with modeling.")
else:
    # Save cleaned merged data
    merged_clean.to_csv('merged_crop_data_clean.csv', index=False)
    print("\nCleaned merged dataset saved to 'merged_crop_data_clean.csv'")

    # ============================================================================
    # STEP 4: PREPARE FOR MODELING
    # ============================================================================
    print("\n" + "=" * 80)
    print("STEP 4: PREPARING DATA FOR MODELING...")
    print("=" * 80)

    # Get all numeric columns
    numeric_cols = merged_clean.select_dtypes(include=[np.number]).columns.tolist()
    print(f"\nNumerical columns ({len(numeric_cols)}):")
    for col in numeric_cols:
        print(f"  - {col}")

    # Identify potential target columns
    production_cols = [col for col in numeric_cols if 'production' in col.lower() or '2010-11' in col or '2011' in col]
    print(f"\nPotential target columns: {production_cols}")

    # ============================================================================
    # STEP 5: BUILD AND TRAIN MODEL
    # ============================================================================
    print("\n" + "=" * 80)
    print("STEP 5: TRAINING MODEL...")
    print("=" * 80)

    # Select target column - MODIFY THIS based on your actual columns
    # Try to find the most recent production column
    target_col = None
    for col in numeric_cols:
        if 'production' in col.lower() and ('2010' in col or '2011' in col):
            target_col = col
            break

    # If no production column found, use the last numeric column
    if target_col is None:
        target_col = numeric_cols[-1] if numeric_cols else None

    if target_col is None:
        print("ERROR: No suitable target column found!")
        print("Available columns:", merged_clean.columns.tolist())
    else:
        print(f"Target column: {target_col}")

        # Define features (all numeric except target)
        feature_cols = [col for col in numeric_cols if col != target_col]
        print(f"Feature columns ({len(feature_cols)}): {feature_cols}")

        # Create X and y
        X = merged_clean[feature_cols]
        y = merged_clean[target_col]

        # Verify no NaN
        print(f"\nNaN in X: {X.isnull().sum().sum()}")
        print(f"NaN in y: {y.isnull().sum()}")

        # Check if we have enough data
        if len(X) < 5:
            print("\nERROR: Not enough data to train model!")
        else:
            # Split data
            test_size = min(0.3, 1.0 - (3.0 / len(X)))  # Ensure at least 3 samples in train
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=test_size, random_state=42
            )

            print(f"\nTrain size: {len(X_train)}")
            print(f"Test size: {len(X_test)}")

            # Train model
            model = LinearRegression()
            model.fit(X_train, y_train)

            # Make predictions
            y_train_pred = model.predict(X_train)
            y_test_pred = model.predict(X_test)

            # Calculate metrics
            train_r2 = r2_score(y_train, y_train_pred)
            train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
            train_mae = mean_absolute_error(y_train, y_train_pred)

            test_r2 = r2_score(y_test, y_test_pred)
            test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
            test_mae = mean_absolute_error(y_test, y_test_pred)

            # ============================================================================
            # STEP 6: DISPLAY RESULTS
            # ============================================================================
            print("\n" + "=" * 80)
            print("MODEL PERFORMANCE:")
            print("=" * 80)

            print(f"\nTraining Set:")
            print(f"  R² Score: {train_r2:.4f}")
            print(f"  RMSE: {train_rmse:.4f}")
            print(f"  MAE: {train_mae:.4f}")

            print(f"\nTest Set:")
            print(f"  R² Score: {test_r2:.4f}")
            print(f"  RMSE: {test_rmse:.4f}")
            print(f"  MAE: {test_mae:.4f}")

            # Feature importance
            print(f"\n" + "=" * 80)
            print("TOP 5 MOST IMPORTANT FEATURES:")
            print("=" * 80)
            feature_importance = pd.DataFrame({
                'Feature': feature_cols,
                'Coefficient': model.coef_
            }).sort_values('Coefficient', key=abs, ascending=False)
            print(feature_importance.head())

            # Save predictions
            results_df = pd.DataFrame({
                'Actual': y_test.values,
                'Predicted': y_test_pred,
                'Error': y_test.values - y_test_pred
            })
            results_df.to_csv('prediction_results.csv', index=False)
            print("\nPredictions saved to 'prediction_results.csv'")

            print("\n" + "=" * 80)
            print("COMPLETED SUCCESSFULLY!")
            print("=" * 80)

In [ ]:
import pandas as pd

# Create a sample input for prediction.
# These values should correspond to the feature columns used in the model.
# You can modify these values to get different predictions.
sample_input = pd.DataFrame({
    '3-2005': [100.0],
    '3-2006': [110.0],
    '3-2007': [120.0],
    '3-2008': [130.0],
    '3-2009': [140.0],
    '3-2010': [150.0],
    '3-2011': [160.0],
    '3-2012': [170.0],
    '3-2013': [180.0],
    '3-2014': [190.0],
    'Production 2006-07': [100.0],
    'Production 2007-08': [110.0],
    'Production 2008-09': [120.0],
    'Production 2009-10': [130.0],
    'Area 2006-07': [50.0],
    'Area 2007-08': [55.0],
    'Area 2008-09': [60.0],
    'Area 2009-10': [65.0],
    'Area 2010-11': [70.0],
    'Yield 2006-07': [10.0],
    'Yield 2007-08': [7.0],
    'Yield 2008-09': [12.0],
    'Yield 2009-10': [13.0],
    'Yield 2010-11': [14.0]
})

# Ensure the columns are in the same order as the training features
sample_input = sample_input[feature_cols]

# Make a prediction
predicted_value = model.predict(sample_input)

print(f"Predicted Production for 2010-11: {predicted_value[0]:.2f}")
